<a href="https://colab.research.google.com/github/rajkumar1325/CODSOFT/blob/main/CreditCardFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#1. loading datasets and visualising
import pandas as pd

#2. Preprocessing
from sklearn.model_selection import train_test_split

#3. model training
from sklearn.linear_model import LogisticRegression

#4. model evaluation
from sklearn.metrics import classification_report, accuracy_score

In [48]:
trainData = pd.read_csv('/content/fraudTrain.csv')
testData = pd.read_csv('/content/fraudTest.csv')


# **Filtering** out the dataset

### checking for training dataset

In [49]:
trainData.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [50]:
trainData.isnull().sum()  #checking for null values

,0
Unnamed: 0,0
trans_date_trans_time,0
cc_num,0
merchant,0
category,0
amt,0
first,0
last,0
gender,0
street,0


In [51]:
trainData['is_fraud'].value_counts()


,count
is_fraud,
0,1289169
1,7506


In [52]:
#  The data is highly imbalanaced and contain alpha-numeric character(String) also


## Checking for testing dataset

In [53]:
testData.shape

(555719, 23)

In [54]:
testData.value_counts('is_fraud')

,count
is_fraud,
0,553574
1,2145


In [55]:
trainData.dtypes


,0
Unnamed: 0,int64
trans_date_trans_time,object
cc_num,int64
merchant,object
category,object
amt,float64
first,object
last,object
gender,object
street,object


In [56]:
testData.dtypes

,0
Unnamed: 0,int64
trans_date_trans_time,object
cc_num,int64
merchant,object
category,object
amt,float64
first,object
last,object
gender,object
street,object


In [57]:
# It also contains string data

### dataset contains **string** data-type also `and model cant be trianed on string.`


In [58]:
# removing all the column that doesn't contain numeric character {as we can train the model on digits only}
trainData = testData.drop(columns= ['trans_num', 'job', 'dob', 'state', 'city', 'street', 'gender', 'last', 'first', 'category', 'merchant', 'trans_date_trans_time'], axis=1)
testData= testData.drop(columns = ['trans_num', 'job', 'dob', 'state', 'city', 'street', 'gender', 'last', 'first', 'category', 'merchant', 'trans_date_trans_time'], axis=1)

# PreProcessing
*making a balanced dataset*

In [59]:
# splitting the dataset
real = trainData[trainData['is_fraud'] == 0]    #legit t/c
fraud = trainData[trainData['is_fraud'] == 1]   #fraud t/c

In [60]:
print("real" ,real.shape)
print("Fraud: ", fraud.shape)

real (553574, 11)
Fraud:  (2145, 11)


In [61]:
# making a sample of datasets equals to the fraud dataset
realSample = real.sample(frac=0.1, random_state=42)
# random_state=42: Ensures reproducibility, so that each time you run the code, the same random sample is selected.

In [62]:
print("real:: ",realSample.shape)
print("fraud:: ",fraud.shape)

real::  (55357, 11)
fraud::  (2145, 11)


# Concatinating both datasets to make a new balanced dataset

In [63]:
newDataset = pd.concat([realSample,fraud], axis=0)

In [64]:
newDataset.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,57502.000000,5.750200e+04,57502.000000,57502.000000,57502.000000,57502.000000,5.750200e+04,5.750200e+04,57502.000000,57502.000000,57502.000000
mean,276290.364439,4.102844e+17,84.100500,48922.792442,38.561557,-90.296494,8.769626e+04,1.380628e+09,38.564399,-90.295473,0.037303
std,160048.635561,1.299460e+18,180.325861,26888.655741,5.062362,13.714963,2.958628e+05,5.192372e+06,5.097473,13.727802,0.189505
min,2.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.669812,0.000000
25%,138135.750000,1.800312e+14,9.870000,26237.000000,34.690200,-96.917700,7.590000e+02,1.376005e+09,34.773056,-96.980312,0.000000
50%,275131.500000,3.518669e+15,48.780000,48206.000000,39.390000,-87.591700,2.456000e+03,1.380639e+09,39.410210,-87.523480,0.000000
75%,414835.500000,4.629452e+15,86.527500,72042.000000,41.940400,-80.175200,2.032800e+04,1.385842e+09,41.963930,-80.238857,0.000000
max,555710.000000,4.992346e+18,12882.330000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.671540,-66.960745,1.000000


In [65]:
newDataset.value_counts('is_fraud')

,count
is_fraud,
0,55357
1,2145


# Splitting the ***train*** data into features and Target

In [66]:
x_train = newDataset.drop(columns='is_fraud', axis=1) #removing the fraud column from dataset
y_train = newDataset['is_fraud']  #storing the removed column(fraud) into new variable

# Splitting the ***test*** data into features and target

In [67]:
testData.shape

(555719, 11)

In [68]:
testData.value_counts('is_fraud')

,count
is_fraud,
0,553574
1,2145


In [69]:
# Splitting
X_test = testData.drop(['is_fraud'], axis=1)
Y_test = testData['is_fraud']

In [70]:
X_test.head()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
0,0,2291163933867244,2.86,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714
1,1,3573030041201292,29.84,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431
2,2,3598215285024754,41.28,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111
3,3,3591919803438423,60.05,32780,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061
4,4,3526826139003047,3.19,49632,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734


# Training
using Logistic Regression

In [71]:
model = LogisticRegression()  #loading lr instance into model

In [72]:
print(x_train.shape)
print(y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(57502, 10)
(57502,)
(555719, 10)
(555719,)


In [73]:
# Training the lr model using train dat
model.fit(x_train,y_train)

LogisticRegression()

# evaluation of the model

In [74]:
# Accuracy score in triaining data
X_trainPrediction = model.predict(x_train)
trainingDataAccuracy = accuracy_score(X_trainPrediction, y_train)

In [75]:
print(trainingDataAccuracy)

0.9626969496713158


In [76]:
# Accuracy score based on test data
X_testPrediction = model.predict(X_test)
testDataAccuracy = accuracy_score(X_testPrediction, Y_test)

In [77]:
testDataAccuracy

0.9961401355721147

# classificaiton report

In [78]:
y_pred = model.predict(X_test)
report = classification_report(Y_test, y_pred)

print(report)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
